In [39]:
# Import necessary libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Dropout,Activation,Conv2D,MaxPooling2D,BatchNormalization,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import os
import seaborn as sns

In [40]:
# Define dictionaries for mapping emotions to integers and vice versa
int2emotions = {0:'angry',1:'fear',2:'happy',3:'heutral',4:'sad',5:'surprise',6:'disgust'}
emotions2int = {'angry':0,'fear':1,'happy':2,'neutral':3,'sad':4,'surprise':5,'disgust':6}

# Data structure to hold image paths, labels, and purpose
dic = {'images':[], 'labels':[], 'purpose':[]}
    
# Iterate through the directory structure to gather image data    
for d in os.listdir('E://TNSDC//model/'):
    for emotion in os.listdir(f'E:/TNSDC/model/{d}'):
        print(emotion)
        for i in os.listdir(f'E:/TNSDC/model/{d}/{emotion}'):
            img = cv2.imread(f'E:/TNSDC/model/{d}/{emotion}/{i}',0)
            img = img.reshape(48,48,1)
            
            dic['images'].append(img)
            dic['labels'].append(emotion)
            
            if d=='train':
                dic['purpose'].append('T')
            else:
                dic['purpose'].append('V')

# Convert dictionary to DataFrame
df = pd.DataFrame(dic)
df.head()

angry
disgust
fear
happy
neutral
sad
surprise
angry
disgust
fear
happy
neutral
sad
surprise


,images,labels,purpose
0,"[[[123], [123], [126], [131], [124], [69], [10...",angry,V
1,"[[[127], [121], [124], [137], [123], [118], [1...",angry,V
2,"[[[255], [255], [255], [255], [255], [255], [2...",angry,V
3,"[[[22], [12], [12], [13], [16], [15], [19], [3...",angry,V
4,"[[[17], [18], [19], [19], [17], [15], [16], [1...",angry,V


In [41]:
# Split data into training and validation sets
train_data = df[df['purpose']=='T']
val_data = df[df['purpose']=='V']

In [42]:

train_data.head()

,images,labels,purpose
5300,"[[[221], [218], [222], [230], [235], [240], [2...",angry,T
5301,"[[[11], [8], [9], [10], [7], [7], [9], [8], [9...",angry,T
5302,"[[[146], [138], [148], [155], [160], [162], [1...",angry,T
5303,"[[[218], [163], [16], [3], [7], [9], [3], [8],...",angry,T
5304,"[[[8], [7], [9], [11], [9], [8], [11], [14], [...",angry,T


In [43]:
val_data.head()

,images,labels,purpose
0,"[[[123], [123], [126], [131], [124], [69], [10...",angry,V
1,"[[[127], [121], [124], [137], [123], [118], [1...",angry,V
2,"[[[255], [255], [255], [255], [255], [255], [2...",angry,V
3,"[[[22], [12], [12], [13], [16], [15], [19], [3...",angry,V
4,"[[[17], [18], [19], [19], [17], [15], [16], [1...",angry,V


In [44]:
train_data['labels'].value_counts()

labels
happy       2772
surprise    1463
neutral     1321
fear        1194
sad         1128
angry       1112
disgust      436
Name: count, dtype: int64

In [45]:
happy_count = (train_data['labels'] == 'happy').sum()
print("Happy class count:", happy_count)

if happy_count > 0:
  happy_df = train_data[train_data['labels'] == 'happy'].sample(n=min(happy_count, 3171))
else:
  happy_df = pd.DataFrame()
neutral_count = (train_data['labels'] == 'neutral').sum()
print("neutral class count:", neutral_count)

if neutral_count > 0:
  neutral_df = train_data[train_data['labels'] == 'neutral'].sample(n=min(neutral_count, 3171))
else:
  neutral_df = pd.DataFrame()
sad_count = (train_data['labels'] == 'sad').sum()
print("sad class count:", sad_count)

if sad_count > 0:
  sad_df = train_data[train_data['labels'] == 'sad'].sample(n=min(sad_count, 3171))
else:
  sad_df = pd.DataFrame()
fear_count = (train_data['labels'] == 'fear').sum()
print("fear class count:", fear_count)

if fear_count > 0:
  fear_df = train_data[train_data['labels'] == 'fear'].sample(n=min(fear_count, 3171))
else:
  fear_df = pd.DataFrame()
angry_count = (train_data['labels'] == 'angry').sum()
print("angry class count:", angry_count)

if angry_count > 0:
  angry_df = train_data[train_data['labels'] == 'angry'].sample(n=min(angry_count, 3171))
else:
  angry_df = pd.DataFrame()
surprise_count = (train_data['labels'] == 'surprise').sum()
print("surprise class count:", surprise_count)

if surprise_count > 0:
  surprise_df = train_data[train_data['labels'] == 'surprise'].sample(n=min(surprise_count, 3171))
else:
  surprise_df = pd.DataFrame()
disgust_count = (train_data['labels'] == 'disgust').sum()
print("disgust class count:", disgust_count)

if disgust_count > 0:
  disgust_df = train_data[train_data['labels'] == 'disgust'].sample(n=min(disgust_count, 3171))
else:
  disgust_df = pd.DataFrame()

train_data = pd.concat([happy_df,neutral_df,sad_df,fear_df,angry_df,surprise_df,disgust_df])

train_data = train_data.sample(frac=1)
train_data.reset_index(inplace=True)
train_data.drop('index',inplace=True,axis=1)

train_data.head()

Happy class count: 2772
neutral class count: 1321
sad class count: 1128
fear class count: 1194
angry class count: 1112
surprise class count: 1463
disgust class count: 436


,images,labels,purpose
0,"[[[191], [174], [157], [157], [162], [162], [1...",sad,T
1,"[[[172], [175], [169], [177], [178], [176], [1...",happy,T
2,"[[[192], [190], [194], [192], [193], [202], [8...",happy,T
3,"[[[5], [6], [7], [18], [25], [27], [28], [21],...",neutral,T
4,"[[[37], [39], [34], [58], [113], [124], [135],...",neutral,T


In [46]:
train_data['labels'].value_counts()

labels
happy       2772
surprise    1463
neutral     1321
fear        1194
sad         1128
angry       1112
disgust      436
Name: count, dtype: int64

In [47]:
batch_size= 32
classes = 7
rows,columns=48,48

In [48]:
# Convert labels to one-hot encoded vectors
train_labels = [emotions2int[label] for label in train_data['labels']]
train_labels = to_categorical(train_labels)

val_labels = [emotions2int[label] for label in val_data['labels']]
val_labels = to_categorical(val_labels)


train_data = list(train_data['images'])
train_data = np.array(train_data)

val_data = list(val_data['images'])
val_data = np.array(val_data)

In [49]:
train_data.shape

(9426, 48, 48, 1)

In [50]:
val_data.shape

(5300, 48, 48, 1)

In [51]:
# Define the CNN architecture
model = Sequential()


model.add(Conv2D(64,(3,3),activation='elu',input_shape=(rows,columns,1),kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),activation='elu',input_shape=(rows,columns,1),kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


model.add(Conv2D(128,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


model.add(Conv2D(256,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


model.add(Conv2D(512,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(512,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


model.add(Flatten())
model.add(Dense(256,activation='elu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(Dense(128,activation='elu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(Dense(64,activation='elu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(Dense(classes,activation='softmax',kernel_initializer='he_normal'))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 48, 48, 64)        640       
                                                                 
 batch_normalization_11 (Ba  (None, 48, 48, 64)        256       
 tchNormalization)                                               
                                                                 
 conv2d_9 (Conv2D)           (None, 48, 48, 64)        36928     
                                                                 
 batch_normalization_12 (Ba  (None, 48, 48, 64)        256       
 tchNormalization)                                               
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 24, 24, 64)        0         
 g2D)                                                            
                                                      

In [54]:

# Define callbacks
checkpoint = ModelCheckpoint('model\\6_class_emotion_detector_V2.h5',
                             save_best_only=True,
                             mode='min',
                             monitor='val_loss',
                             verbose=1,
                             options=None)

earlystopping = EarlyStopping(patience=10,
                             verbose=1,
                             min_delta=0,
                             monitor='val_loss',
                             restore_best_weights=True)


callbacks = [checkpoint, earlystopping]

# Compile the model
model.compile(metrics=['accuracy'],
             optimizer='rmsprop',
             loss='categorical_crossentropy')

train_samples = 28273
validation_samples = 3534
batch_size = 64
epochs=11

history = model.fit(train_data,
                    train_labels,
                    epochs=epochs,
                    steps_per_epoch=train_samples//batch_size,
                    validation_data=(val_data,val_labels),
                    validation_steps=validation_samples//batch_size,
                    callbacks=callbacks)

Epoch 1/11
441/441 [==============================] - ETA: 0s - loss: 1.6570 - accuracy: 0.3922
Epoch 1: val_loss improved from inf to 1.51394, saving model to model\6_class_emotion_detector_V2.h5


c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


441/441 [==============================] - 774s 2s/step - loss: 1.6570 - accuracy: 0.3922 - val_loss: 1.5139 - val_accuracy: 0.4400
Epoch 2/11
441/441 [==============================] - ETA: 0s - loss: 1.4983 - accuracy: 0.4402
Epoch 2: val_loss improved from 1.51394 to 1.43005, saving model to model\6_class_emotion_detector_V2.h5
441/441 [==============================] - 856s 2s/step - loss: 1.4983 - accuracy: 0.4402 - val_loss: 1.4300 - val_accuracy: 0.4704
Epoch 3/11
441/441 [==============================] - ETA: 0s - loss: 1.4164 - accuracy: 0.4756
Epoch 3: val_loss improved from 1.43005 to 1.38698, saving model to model\6_class_emotion_detector_V2.h5
441/441 [==============================] - 904s 2s/step - loss: 1.4164 - accuracy: 0.4756 - val_loss: 1.3870 - val_accuracy: 0.4785
Epoch 4/11
441/441 [==============================] - ETA: 0s - loss: 1.3398 - accuracy: 0.5025
Epoch 4: val_loss improved from 1.38698 to 1.29662, saving model to model\6_class_emotion_detector_V2.h5
4

In [55]:
# Detection of emotions 

int2emotions = {0: 'Angry', 1: 'Fear', 2: 'Happy', 3: 'Neutral', 4: 'Sad', 5: 'Surprise', 6: 'Disgust'}
emotion_text_colors = {'Angry': (0, 0, 255), 'Fear': (0, 128, 255), 'Happy': (0, 255, 0),
                       'Neutral': (255, 255, 0), 'Sad': (255, 0, 0), 'Surprise': (255, 165, 0),
                       'Disgust': (128, 0, 128)}
emotion_bg_colors = {'Angry': (255, 255, 255), 'Fear': (255, 255, 255), 'Happy': (255, 255, 255),
                     'Neutral': (255, 255, 255), 'Sad': (255, 255, 255), 'Surprise': (255, 255, 255),
                     'Disgust': (255, 255, 255)}

model = load_model('model/6_class_emotion_detector_V2.h5') 

classifier = cv2.CascadeClassifier(r"E:/TNSDC/haarcascade_frontalface_default.xml")

def add_transparent_text_bg(img, text, pos, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=1, font_thickness=2, text_color=(0, 0, 0), bg_color=(255, 255, 255), bg_alpha=0.5):
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_x, text_y = pos
    overlay = img[text_y:text_y + text_size[1] + 2, text_x:text_x + text_size[0] + 2]
    cv2.putText(img, text, pos, font, font_scale, text_color, font_thickness)

def detect_face(frame):
    if classifier.empty():
        print("Error: Could not load face cascade classifier.")
        return frame

    faces = classifier.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=4)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 255), 2) 

        face_roi = frame[y:y+h, x:x+w]
        face_roi = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)  
        face_roi = cv2.resize(face_roi, (48, 48)) 
        face_roi = face_roi.astype('float32') / 255.0 
        face_roi = np.expand_dims(face_roi, axis=0) 

        predicted_emotion = int2emotions[np.argmax(model.predict(face_roi))]
        text_color = emotion_text_colors[predicted_emotion]
        bg_color = emotion_bg_colors[predicted_emotion]

        add_transparent_text_bg(frame, predicted_emotion, (x, y - 30), text_color=text_color, bg_color=bg_color)

    return frame

cap = cv2.VideoCapture(0)

cv2.namedWindow('Emotion Detector', cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty('Emotion Detector', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while True:
    ret, frame = cap.read()
    if ret:
        frame = detect_face(frame)
        cv2.imshow('Emotion Detector', frame)
        if cv2.waitKey(1) == 27:
            break

cap.release()
cv2.destroyAllWindows()




1/1 [==============================] - 0s 35ms/step
